# Metric 1 : Quarterly comparison of metadata on files with a common template, such as the information template and artwork template

In continuation to the numbers we provided in T231952-part-1a for files that have common templates, we will look into getting number of files that have structured data.

## Investigating claim actions

Exploring the data for claims/labels in order to understand whether we can identify the number of statements a given file might have based on edit comments.

In [5]:
import pandas as pd
import numpy as np

import datetime as dt

from wmfdata import hive, mariadb

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


## Configuration variables

In [6]:
wmf_snapshot = '2019-11'
start_date = '2019-01-01' # first date of caption edits
start_date_new = '2019-07-31' # first date of caption edits
end_date = '2019-12-01' # last date of caption edits (exclusive)

## Investigating claim actions

I dug into the edit comments a bit to find examples, and found that [this file](https://commons.wikimedia.org/wiki/File:Rosendahl,_Darfeld,_Ortsansicht_--_2014_--_9391.jpg) has a bunch of property edits that provided good insight into what to look for. Basically, it meant that I had to expand my search to anything that starts with "wb". Using that insight, the below query looks for an edit comment matching "wb{something}-{something}:" (anchored at the start of the comment with space for "/* ") and aggregates over the first and second "something".

In [18]:
claim_query = '''
SELECT claim_action, claim_subaction, count(*) AS num_actions
FROM (
    SELECT
        regexp_extract(event_comment, "^...(wb[^-]+)", 1) AS claim_action,
        regexp_extract(event_comment, "^...wb[^-]+-([^:]+):", 1) AS claim_subaction
    FROM wmf.mediawiki_history
    WHERE snapshot = "{snapshot}"
    AND wiki_db = "commonswiki"
    AND event_entity = "revision"
    AND event_type = "create"
    AND event_timestamp >= "{start_date}"
    AND event_timestamp < "{end_date}"
    AND page_is_deleted = false -- only count live pages
    AND page_namespace = 6 -- only count files
    AND event_comment REGEXP "^...(wb[^-]+)-([^:]+):"
) AS ce
GROUP BY claim_action, claim_subaction
'''

In [19]:
claim_counts = hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        claim_query.format(
            snapshot = wmf_snapshot,
            start_date = start_date,
            end_date = end_date
        )
    ]
)

In [28]:
claim_counts.sort_values(['claim_action', 'claim_subaction'])

,claim_action,claim_subaction,num_actions
2,wbcreateclaim,create,1655105
8,wbeditentity,update,261794
0,wbeditentity,update-languages,872
11,wbremoveclaims,remove,22021
3,wbremoveclaims,update,14259
5,wbsetclaim,create,537587
6,wbsetclaim,update,40832
9,wbsetdescription,add,2
10,wbsetlabel,add,1631793
7,wbsetlabel,remove,11321


From Morten's analysis on [Media files containing structured fields in non-English languages](https://github.com/wikimedia-research/SDC-metrics-2019/blob/master/T231952-part-3.ipynb) we know that wbsetlabel is used for caption additions. The wbremoveclaims indicates if a single property or multiple properties have been removed from a file.    
Thus, we can conclude that the following comments are used to add or update properties on a file. In other words, to know how rich is the metadata of files on Commons we can consider the following comments:  
- wbcreateclaim-create   
- wbeditentity-update   
- wbsetclaim-create   
- wbsetclaim-update   

Also, from the example file, we know that a single edit can modify multiple properties, meaning that we cannot know how many properties a file has based on the comments. 

## Claim actions from Aug 1 onwards
The edit comments are an ideal way to know which properties have been added/updated/removed from a file. However, we found that information about properties were not added to the comments for the time frame we needed i.e. beginning January 2019. 

During the meeting with Amanda and Ramsey on 12-10-19, we raised the issue of not being able to use edit comments to see the properties that were added to a file. Per Ramsey, properties started appearing on edit comments from July 31, 2019 when UI went live. Hence we have decided to go forward with 2 approaches:

1. Product Analytics will provide numbers from when the edit comments started getting properties (i.e. Aug 1st 2019 when UI went live) by 12/20 or before. This will be used in the grant proposal that is due on Jan 10, 2020.
2. Product Analytics to use Commons data dump to get numbers from Jan 2019 that are required for the Grant proposal to Sloan Foundation (due Feb 28, 2020 ). @Abit will reach out to Ariel for making the dump available (for us to parse as well as in Query service). We can also include common template data in the dump for metadata comparison.

### Claim actions for all wb comments
***All the analysis from this point onwards will consider the dates from 2019-07-31 to 2019-12-01***

In [4]:
claim_query_new = '''
SELECT claim_action, claim_subaction, count(*) AS num_actions
FROM (
    SELECT
        regexp_extract(event_comment, "^...(wb[^-]+)", 1) AS claim_action,
        regexp_extract(event_comment, "^...wb[^-]+-([^:]+):", 1) AS claim_subaction
    FROM wmf.mediawiki_history
    WHERE snapshot = "{snapshot}"
    AND wiki_db = "commonswiki"
    AND event_entity = "revision"
    AND event_type = "create"
    AND event_timestamp >= "{start_date_new}"
    AND event_timestamp < "{end_date}"
    AND page_is_deleted = false -- only count live pages
    AND page_namespace = 6 -- only count files
    AND event_comment REGEXP "^...(wb[^-]+)-([^:]+):"
) AS ce
GROUP BY claim_action, claim_subaction
'''

In [7]:
claim_counts_new = hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        claim_query_new.format(
            snapshot = wmf_snapshot,
            start_date_new = start_date_new,
            end_date = end_date
        )
    ]
)

In [8]:
claim_counts_new.sort_values(['claim_action', 'claim_subaction'])

,claim_action,claim_subaction,num_actions
2,wbcreateclaim,create,1066714
8,wbeditentity,update,169234
0,wbeditentity,update-languages,872
10,wbremoveclaims,remove,7976
3,wbremoveclaims,update,472
5,wbsetclaim,create,359941
6,wbsetclaim,update,32370
9,wbsetlabel,add,575205
7,wbsetlabel,remove,3203
1,wbsetlabel,set,15285


### Claim actions for wbcreateclaim-create, wbeditentity-update, wbsetclaim-create, and wbsetclaim-update  comments


In [3]:
# Number of comments that have the wbcreateclaim-create, wbeditentity-update, wbsetclaim-create, and wbsetclaim-update 

wbadditions_detail_query = '''
SELECT claim_action, claim_subaction, count(*) AS num_actions
FROM (
    SELECT
        regexp_extract(event_comment, "^...(wb[^-]+)", 1) AS claim_action,
        regexp_extract(event_comment, "^...wb[^-]+-([^:]+):", 1) AS claim_subaction
    FROM wmf.mediawiki_history
    WHERE snapshot = "{snapshot}"
    AND wiki_db = "commonswiki"
    AND event_entity = "revision"
    AND event_type = "create"
    AND event_timestamp >= "{start_date_new}"
    AND event_timestamp < "{end_date}"
    AND page_is_deleted = false -- only count live pages
    AND page_namespace = 6 -- only count files
    AND (event_comment REGEXP "^...wbcreateclaim-create:"  
OR event_comment REGEXP "^...wbeditentity-update:"
OR event_comment REGEXP "^...wbsetclaim-create:"
OR event_comment REGEXP "^...wbsetclaim-update:")
) AS ce
GROUP BY claim_action, claim_subaction
'''

In [4]:
wbadditions_detail_count = hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        wbadditions_detail_query.format(
            snapshot = wmf_snapshot,
            start_date_new = start_date_new,
            end_date = end_date
        )
    ]
)

In [5]:
wbadditions_detail_count

,claim_action,claim_subaction,num_actions
0,wbcreateclaim,create,1066809
1,wbsetclaim,create,361534
2,wbsetclaim,update,32447
3,wbeditentity,update,172925


### Files with property additions 


In [4]:
# Number of files with wbcreateclaim-create, wbeditentity-update, wbsetclaim-create, and wbsetclaim-update 

wb_additions_count_query = '''
SELECT COUNT(DISTINCT page_id) AS num_pages
FROM wmf.mediawiki_history
WHERE snapshot = "{snapshot}"
AND wiki_db = "commonswiki"
AND event_entity = "revision"
AND event_type = "create"
AND event_timestamp >= "{start_date_new}"
AND event_timestamp < "{end_date}"
AND page_is_deleted = false -- only count live pages
AND page_namespace = 6 -- only count files
AND (event_comment REGEXP "^...wbcreateclaim-create:"  
OR event_comment REGEXP "^...wbeditentity-update:"
OR event_comment REGEXP "^...wbsetclaim-create:"
OR event_comment REGEXP "^...wbsetclaim-update:"
)
'''

In [6]:
wb_additions_count = hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        wb_additions_count_query.format(
            snapshot = wmf_snapshot,
            start_date_new = start_date_new,
            end_date = end_date
        )
    ]
)

We get around 1.27 million Commons files that have metadata from July 31 to Nov 30

In [7]:
wb_additions_count

,num_pages
0,1270249


### Files with specific Structured data items added to them
We are interested in getting the numbers of files with the following properties added to them  
1. Caption P2096   
2. Date of Creation P571    
3. Date of Publication P577  
4. Creator P170  
5. License P275   
6. Digital Representation of P6243  
7. Depicts P180 

In [33]:
#Number of Files with Captions (P2096) 

wb_caption_count_query="""
SELECT COUNT(DISTINCT page_id) AS num_pages_captions
FROM wmf.mediawiki_history
WHERE snapshot = "{snapshot}"
AND wiki_db = "commonswiki"
AND event_entity = "revision"
AND event_type = "create"
AND event_timestamp >= "{start_date_new}"
AND page_is_deleted = false -- only count live pages
AND page_namespace = 6 -- only count files
AND (event_comment REGEXP "^...wbcreateclaim-create:"
OR event_comment REGEXP "^...wbeditentity-update:"
OR event_comment REGEXP "^...wbsetclaim-create:"
OR event_comment REGEXP "^...wbsetclaim-update:"
)
AND event_comment LIKE "%P2096%"
"""

In [34]:
wb_caption_count = hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        wb_caption_count_query.format(
            snapshot = wmf_snapshot,
            start_date_new = start_date_new
           
        )
    ]
)


We believe the low count in Captions could be because captions are better captured using wbsetlabel comments. 

In [35]:
wb_caption_count

,num_pages_captions
0,767


Recalculating captions using the  wbsetlabel-add,set and remove comments

In [9]:
caption_query = '''
SELECT COUNT(DISTINCT page_id) AS num_pages_captions
FROM wmf.mediawiki_history
WHERE snapshot = "{snapshot}"
AND wiki_db = "commonswiki"
AND event_entity = "revision"
AND event_type = "create"
AND event_timestamp >= "2019-07-31"
AND event_comment REGEXP "^...wbsetlabel-(add|set):"
'''

In [10]:
label_counts = hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        caption_query.format(
            snapshot = wmf_snapshot,
            
        )
    ]
)


In [11]:
label_counts

,num_pages_captions
0,510700


In [36]:
#Number of Files with Date of Creation (P571) 

wb_date_of_creation_query="""
SELECT COUNT(DISTINCT page_id) AS num_pages_creation_date
FROM wmf.mediawiki_history
WHERE snapshot = "{snapshot}"
AND wiki_db = "commonswiki"
AND event_entity = "revision"
AND event_type = "create"
AND event_timestamp >= "2019-07-31"
AND event_timestamp < "2019-12-01"
AND page_is_deleted = false -- only count live pages
AND page_namespace = 6 -- only count files
AND (event_comment REGEXP "^...wbcreateclaim-create:"
OR event_comment REGEXP "^...wbeditentity-update:"
OR event_comment REGEXP "^...wbsetclaim-create:"
OR event_comment REGEXP "^...wbsetclaim-update:"
)
AND
event_comment like "%P571%"
"""

In [37]:
wb_date_of_creation_count= hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        wb_date_of_creation_query.format(
            snapshot = wmf_snapshot 
        )
    ]
)


In [38]:
wb_date_of_creation_count

,num_pages_creation_date
0,9214


In [39]:
#Number of Files with Date of Publication (P577) 

wb_date_of_publication_query="""
SELECT COUNT(DISTINCT page_id) AS num_pages_publication_date
FROM wmf.mediawiki_history
WHERE snapshot = "{snapshot}"
AND wiki_db = "commonswiki"
AND event_entity = "revision"
AND event_type = "create"
AND event_timestamp >= "2019-07-31"
AND event_timestamp < "2019-12-01"
AND page_is_deleted = false -- only count live pages
AND page_namespace = 6 -- only count files
AND (event_comment REGEXP "^...wbcreateclaim-create:"
OR event_comment REGEXP "^...wbeditentity-update:"
OR event_comment REGEXP "^...wbsetclaim-create:"
OR event_comment REGEXP "^...wbsetclaim-update:"
)
AND
event_comment like "%P577%"
"""

In [40]:
wb_date_of_publication_count= hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        wb_date_of_publication_query.format(
            snapshot = wmf_snapshot 
        )
    ]
)
wb_date_of_publication_count

,num_pages_publication_date
0,0


In [44]:
# Number of files with Creator (P170)

wb_creator_query = '''
SELECT COUNT(DISTINCT page_id) AS num_pages_creator
FROM wmf.mediawiki_history
WHERE snapshot = "{snapshot}"
AND wiki_db = "commonswiki"
AND event_entity = "revision"
AND event_type = "create"
AND event_timestamp >= "2019-07-31"
AND event_timestamp < "2019-12-01"
AND page_is_deleted = false -- only count live pages
AND page_namespace = 6 -- only count files
AND (event_comment REGEXP "^...wbcreateclaim-create:"
OR event_comment REGEXP "^...wbeditentity-update:"
OR event_comment REGEXP "^...wbsetclaim-create:"
OR event_comment REGEXP "^...wbsetclaim-update:"
)
AND event_comment LIKE "%P170%"
'''                              

In [45]:
wb_creator_count= hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        wb_creator_query.format(
            snapshot = wmf_snapshot 
        )
    ]
)
wb_creator_count

,num_pages_creator
0,59920


In [46]:
# Number of files with License (P275)
wb_license_query='''
SELECT COUNT(DISTINCT page_id) AS num_pages_license
FROM wmf.mediawiki_history
WHERE snapshot = "{snapshot}"
AND wiki_db = "commonswiki"
AND event_entity = "revision"
AND event_type = "create"
AND event_timestamp >= "2019-07-31"
AND event_timestamp < "2019-12-01"
AND page_is_deleted = false -- only count live pages
AND page_namespace = 6 -- only count files
AND (event_comment REGEXP "^...wbcreateclaim-create:"
OR event_comment REGEXP "^...wbeditentity-update:"
OR event_comment REGEXP "^...wbsetclaim-create:"
OR event_comment REGEXP "^...wbsetclaim-update:"
)
AND
event_comment like "%P275%"
'''

In [47]:
wb_license_count= hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        wb_license_query.format(
            snapshot = wmf_snapshot 
        )
    ]
)
wb_license_count

,num_pages_license
0,40146


In [48]:
# Number of files with Digital Representation property (P6243)

wb_Digital_Representation_query='''
SELECT COUNT(DISTINCT page_id) AS num_pages_digital_rep
FROM wmf.mediawiki_history
WHERE snapshot = "{snapshot}"
AND wiki_db = "commonswiki"
AND event_entity = "revision"
AND event_type = "create"
AND event_timestamp >= "2019-07-31"
AND event_timestamp < "2019-12-01"
AND page_is_deleted = false -- only count live pages
AND page_namespace = 6 -- only count files
AND (event_comment REGEXP "^...wbcreateclaim-create:"
OR event_comment REGEXP "^...wbeditentity-update:"
OR event_comment REGEXP "^...wbsetclaim-create:"
OR event_comment REGEXP "^...wbsetclaim-update:"
)
AND
event_comment like "%P6243%"
'''

In [49]:
wb_Digital_Representation_count= hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        wb_Digital_Representation_query.format(
            snapshot = wmf_snapshot 
        )
    ]
)
wb_Digital_Representation_count

,num_pages_digital_rep
0,16616


In [50]:
# Number of files with Depicts property (P180)

wb_Depicts_query='''
SELECT COUNT(DISTINCT page_id) AS num_pages_Depicts
FROM wmf.mediawiki_history
WHERE snapshot = "{snapshot}"
AND wiki_db = "commonswiki"
AND event_entity = "revision"
AND event_type = "create"
AND event_timestamp >= "2019-07-31"
AND event_timestamp < "2019-12-01"
AND page_is_deleted = false -- only count live pages
AND page_namespace = 6 -- only count files
AND (event_comment REGEXP "^...wbcreateclaim-create:"
OR event_comment REGEXP "^...wbeditentity-update:"
OR event_comment REGEXP "^...wbsetclaim-create:"
OR event_comment REGEXP "^...wbsetclaim-update:"
)
AND
event_comment like "%P180%"

'''

In [51]:
wb_Depicts_count= hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        wb_Depicts_query.format(
            snapshot = wmf_snapshot 
        )
    ]
)
wb_Depicts_count

,num_pages_depicts
0,868276


### Impact of Property Removals on our analysis

In the above analysis we have only looked at additions and updates. However there are files with properties removed. The wbremoveclaims indicates if a single property or multiple properties have been removed from a file. Based on [Morten's analysis posted on this notebook](https://github.com/wikimedia-research/SDC-metrics-2019/blob/master/wbremoveclaims-exploration.txt), we find that relative to the >1.5M edits that appear to add one
(or more) statements, the removal of multiple statements will not have a significant impact on our
estimates.   
**Hence we should be okay to report the above numbers without including property removals. **